Import PySpark SQL Functions as F

In [0]:
from pyspark.sql import functions as F

catalog = "prime"
schema  = "hospital_management"
volume  = "hm_staff"


Store File Location

In [0]:
file_path = "/Volumes/prime/hospital_management/hm_staff"

Read CSV File into a Dataframe

In [0]:
df_raw= spark.read.csv("/Volumes/prime/hospital_management/hm_staff", header=True, inferSchema=True)

Show DataFrame structure

In [0]:
df_raw.printSchema()

Show first 5 rows of the DataFrame

In [0]:
df_raw.show(5)

Cast columns to clean Data types

In [0]:
df = (
 df_raw
  .withColumn("staff_id", F.col("staff_id").cast("string")) \
  .withColumn("staff_name", F.col("staff_name").cast("string")) \
  .withColumn("role", F.col("role").cast("string")) \
  .withColumn("service", F.col("service").cast("string"))
)

In [0]:
df.printSchema()

Save DataFrame as a Delta Table

In [0]:
df.write.mode("overwrite").saveAsTable("prime.hospital_management.hm_staff")


In [0]:
df = spark.table("prime.hospital_management.hm_staff")
df.show(10)

In [0]:
df.count()

ANALYTICAL QUESTION 1: How many staff members are assigned to each service?

In [0]:

df_service_count = (
    df
        .groupBy("service")
        .agg(F.countDistinct("staff_id").alias("staff_count"))
        .orderBy("staff_count", ascending=False)
)

df_service_count.show()

In [0]:
%sql
--How many staff members are assigned to each service?
SELECT 
  service,
  COUNT(DISTINCT staff_id) AS staff_count
FROM prime.hospital_management.hm_staff
GROUP BY service
ORDER BY staff_count DESC;

ANALYTICAL QUESTION 2: What is the staff distribution by role within each service?

In [0]:

df_role_distribution = (
    df
        .groupBy("service", "role")
        .agg(F.count("staff_id").alias("total_staff"))
        .orderBy(F.col("service").desc(), F.col("total_staff").desc())
)

df_role_distribution.show()

In [0]:
%sql
--What is the staff distribution by role within each service?
SELECT 
    service,
    role,
    COUNT(staff_id) AS total_staff
FROM prime.hospital_management.hm_staff
GROUP BY service, role
ORDER BY service DESC, total_staff DESC;